# Collision Avoidance - Live Demo

In this notebook we'll use the model we trained to detect whether the robot is ``free`` or ``blocked`` to enable a collision avoidance behavior on the robot.  

## Load the trained model

We'll assumed that you've already downloaded the ``best_model.pth`` to your workstation as instructed in the training notebook.  Now, you should upload this model into this notebook's
directory by using the Jupyter Lab upload tool.  Once that's finished there should be a file named ``best_model.pth`` in this notebook's directory.  

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model.  This should look very familiar from the training notebook.

In [1]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 3)

Next, load the trained weights from the ``best_model.pth`` file that you uploaded

In [2]:
model.load_state_dict(torch.load('../../Models/best_model3.pth'))

<All keys matched successfully>

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [3]:
device = torch.device('cuda')
model = model.to(device)

### Create the preprocessing function

We have now loaded our model, but there's a slight issue.  The format that we trained our model doesnt *exactly* match the format of the camera.  To do that, 
we need to do some *preprocessing*.  This involves the following steps

1. Convert from BGR to RGB
2. Convert from HWC layout to CHW layout
3. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4. Transfer the data from CPU memory to GPU memory
5. Add a batch dimension

In [4]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Great! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera.  You should be pretty familiar with this by now.  We'll also create a slider that will display the
probability that the robot is blocked.

In [5]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

In [6]:
camera = Camera.instance(width=224, height=224)

In [7]:
image = widgets.Image(format='jpeg', width=224, height=224)
pao_slider = widgets.FloatSlider(description='pao', min=0.0, max=1.0, orientation='vertical')
move_slider = widgets.FloatSlider(description='env', min=0.0, max=1.0, orientation='vertical')
hand_slider = widgets.FloatSlider(description='hand', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(widgets.HBox([image, pao_slider,move_slider, hand_slider]))

We'll also create our robot instance which we'll need to drive the motors.

In [8]:
from jetbot import Robot

robot = Robot()

Next, we'll create a function that will get called whenever the camera's value changes.  This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. While the neural network output indicates we're blocked, we'll turn left, otherwise we go forward.

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing. 

We accomplish that with the ``observe`` function.

> WARNING: This code will move the robot!! Please make sure your robot has clearance.  The collision avoidance should work, but the neural
> network is only as good as the data it's trained on!

In [9]:
import torch.nn.functional as F
import time
import playsound
import sys 
import multiprocessing
import time
#from threading import Thread

In [18]:
def loose_sequence():
    pW = multiprocessing.Process(target=playsound.playsound, args=("../Music/game_over.mp3",))
    pW.start()
    robot.left(0.1)
    while pW.is_alive():
        i = 1
    robot.stop()

In [19]:
def win_sequence():
    pW = multiprocessing.Process(target=playsound.playsound, args=("../Music/win.mp3",))
    pW.start()
    robot.left(0.1)
    while pW.is_alive():
        i = 1
    robot.stop()

In [20]:
def detect_hand():
    x = preprocess(camera.value)
    y = model(x)
    y = F.softmax(y, dim=1)
    
    prob_hand = float(y.flatten()[0])
    hand_slider.value = prob_hand
    
    if prob_hand > 0.9:
        pG.terminate()
        win_sequence()
        #time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
        return 1
    else:
        return 0

In [21]:
def grab_contours(cnts):
    # if the length the contours tuple returned by cv2.findContours
    # is '2' then we are using either OpenCV v2.4, v4-beta, or
    # v4-official
    if len(cnts) == 2:
        cnts = cnts[0]

    # if the length of the contours tuple is '3' then we are using
    # either OpenCV v3, v4-pre, or v4-alpha
    elif len(cnts) == 3:
        cnts = cnts[1]

    # otherwise OpenCV has changed their cv2.findContours return
    # signature yet again and I have no idea WTH is going on
    else:
        raise Exception(("Contours tuple must have length 2 or 3, "
            "otherwise OpenCV changed their cv2.findContours return "
            "signature yet again. Refer to OpenCV's documentation "
            "in that case"))

    # return the actual contours array
    return cnts

In [31]:
#import imutils
def detect_movement():
    firstFrame = camera.value
    firstFrame = cv2.cvtColor(firstFrame, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(firstFrame, cv2.COLOR_BGR2GRAY)
    firstFrame = cv2.GaussianBlur(gray, (33, 33), 0)
    start_time = time.time()
    while True:
        frame = camera.value
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        text = "Static"
        if frame is None:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (33, 33), 0)
        frameDelta = cv2.absdiff(firstFrame, gray)
        thresh = cv2.threshold(frameDelta, 50, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.dilate(thresh, None, iterations=2)
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)
        cnts = grab_contours(cnts)
        for c in cnts:
            if cv2.contourArea(c) < 85:
                continue
            text = "Movement"
            if text == "Movement":
                return "LOSER"
        if text == "Movement":
            return "LOSER"
        if time.time() >= start_time + 5:
            return "CONTINUE"

In [39]:

while True:
    robot.stop()
    x = preprocess(camera.value)
    y_aux = model(x)
    y = F.softmax(y_aux, dim=1)
    
    prob_pao = float(y.flatten()[2])
    prob_move = float(y.flatten()[1])
    prob_hand = float(y.flatten()[0])
    pao_slider.value = prob_pao
    move_slider.value = prob_move
    hand_slider.value = prob_hand
    
    if prob_pao > 0.94:# and prob_move < 0.1:
        pG = multiprocessing.Process(target=playsound.playsound, args=("../Music/greenlight.mp3",))
        pG.start()
        while pG.is_alive():
            win = detect_hand()
            if win == 1:
                print(y_aux)
                print(y)
                print("end game")
                sys.exit()#break
        robot.right(0.15)
        time.sleep(1.19)
        robot.stop()
        time.sleep(0.6)
        if detect_movement() == "LOSER":
            loose_sequence()
            break #sys.exit()
        else:
            robot.right(0.15)
            time.sleep(1.14)
    elif prob_hand > 0.99:# and prob_move < 0.05:
        print(y_aux)
        print(y)
        win_sequence()
        break #sys.exit()
    else:
        robot.right(0.12)

tensor([[ 4.2239, -0.8142, -3.0691]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[9.9288e-01, 6.4402e-03, 6.7545e-04]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)


In [16]:
robot.stop()